# PART 1: Getting Authors and Titles from Tabula-exported TSV Data

The object: go from a list of tsv filenames (maybe I'll bin them by year?) to an array of [date, rank, weeks on list, title, author(s)]


In [65]:
f_filen = "tabula-1963-03-17.tsv"
nf_filen = "tabula-1963-03-17-nf.tsv"
#How far into the tsv file the data shows up...
data_i = 2
fields = ['rank','title+author','rank_last_week','weeks_on_list']

import csv
from datetime import datetime

def empty(row):
    for r in row:
        if r!="":
            return False
    return True 

def extract_date(datestr):
    print datestr
    date = datetime.strptime(datestr[0],'%B %d, %Y')
    print date
    return date

##Indexes into row data_i of a tsv and then reads it into an array of arrays
def read_tsv(name):
    i = 0
    arr = []
    date = ""
    with open(name) as csvfile:
        reader = csv.reader(csvfile, delimiter='\t')
        for row in reader:
            if i ==0 :
                date = extract_date([row[1]])
            if ((i > data_i) & (not empty(row))):
                arr.append([date]+row)
            i = i+1
        return arr
    
commasv = read_tsv(nf_filen)
print commasv

['March 17, 1963']
1963-03-17 00:00:00
[[datetime.datetime(1963, 3, 17, 0, 0), '1', 'TRAVELS WITH CHARLEY, by John Steinbeck.', '1', '32'], [datetime.datetime(1963, 3, 17, 0, 0), '2', 'HAPPINESS IS A WARM PUPPY, by Charles M. Schulz.', '2', '13'], [datetime.datetime(1963, 3, 17, 0, 0), '3', 'FINAL VERDICT, by Adela Rogers St. Johns.', '4', '27'], [datetime.datetime(1963, 3, 17, 0, 0), '4', 'THE WHOLE TRUTH AND NOTHING BUT, by Hedda Hopper.', '7', '2'], [datetime.datetime(1963, 3, 17, 0, 0), '5', 'O YE JIGS & JULEPS!, by Virginia Cary Hudson.', '3', '43'], [datetime.datetime(1963, 3, 17, 0, 0), '6', 'THE FIRE NEXT TIME, by James Baldwin', '6', '3'], [datetime.datetime(1963, 3, 17, 0, 0), '7', 'SILENT SPRING, by Rachel Carson.', '5', '24'], [datetime.datetime(1963, 3, 17, 0, 0), '8', 'MY LIFE IN COURT, by Louis Nizer.', '10', '68'], [datetime.datetime(1963, 3, 17, 0, 0), '9', 'POINTS OF MY COMPASS, by E.B. White.', '8', '5'], [datetime.datetime(1963, 3, 17, 0, 0), '10', 'THE FALL OF THE 

In [66]:
import os
path = "1963/fiction/"
sixtythree = []
for f in os.listdir(path):
    commasv = read_tsv(path+f)
    sixtythree.append(commasv)
    
print sixtythree[4:8]

['March 17, 1963']
1963-03-17 00:00:00
['May 12, 1963']
1963-05-12 00:00:00
['March 31, 1963']
1963-03-31 00:00:00
['May 5, 1963']
1963-05-05 00:00:00
['May 26, 1963']
1963-05-26 00:00:00
['March 24, 1963']
1963-03-24 00:00:00
['April 7, 1963']
1963-04-07 00:00:00
['April 14, 1963']
1963-04-14 00:00:00
['April 28, 1963']
1963-04-28 00:00:00
['May 19, 1963']
1963-05-19 00:00:00
['April 21, 1963']
1963-04-21 00:00:00
[[[datetime.datetime(1963, 5, 26, 0, 0), '1', 'THE GLASS BLOWERS, by Daphne du Maurier', '1', '8'], [datetime.datetime(1963, 5, 26, 0, 0), '2', 'SEVEN DAYS IN MAY, by Fletcher Knebel and Charles W. Bailey II.', '2', '35'], [datetime.datetime(1963, 5, 26, 0, 0), '3', 'RAISE HIGH THE ROOF BEAM, CARPENTERS, by J.D. Salinger', '3', '15'], [datetime.datetime(1963, 5, 26, 0, 0), '4', 'GRANDMOTHER AND THE PRIESTS, by Taylor Caldwell.', '4', '8'], [datetime.datetime(1963, 5, 26, 0, 0), '5', 'THE SAND PEBBLES, by Richard McKenna.', '5', '19'], [datetime.datetime(1963, 5, 26, 0, 0), '

In [67]:
print "hello world"

hello world


In [3]:
sample_title_auth = commasv[2][2]
#print sample_title_auth

#index into phrase to skip " by"
auth_i = 4

def title_author_pair(data_str):
    split_str = data_str.split(',')
    title = ""
    authors = set()
    i=0
    for phrase in split_str:
        if i == 0:
            title = phrase
        else:
            if phrase.isupper():
                title = title + ", "+phrase
            else:
                author_ph = phrase[auth_i:]
                if " and " in author_ph:
                    author_arr = author_ph.split(" and ")
                    for author in author_arr:
                        if author[len(author)-1]=='.':
                            author = author[:len(author)-1] # strip trailing period
                        authors.add(author)
                else:
                    author = author_ph
                    if author[len(author)-1]=='.':
                        author = author[:len(author)-1] # strip trailing period
                    authors.add(author)
        i = i+1
    return (title,authors)
        
print title_author_pair(sample_title_auth)

('THE SAND PEBBLES', set(['Richard McKenna']))


In [4]:
for row in commasv:
    print(title_author_pair(row[2]))

('RAISE HIGH THE ROOF BEAM,  CARPENTERS', set(['J.D. Salinger']))
('SEVEN DAYS IN MAY', set(['Fletcher Knebel', 'Charles W. Bailey II']))
('THE SAND PEBBLES', set(['Richard McKenna']))
('FAIL-SAFE', set(['Eugene Burdick', 'Harvey Wheeler']))
('THE MOONSPINNERS', set(['Mary Stewart']))
('TRIUMPH', set(['Philip Wylie']))
('A SHADE OF DIFFERENCE', set(['Allen Drury']))
('$100 MISUNDERSTANDING', set(['Robert Gover']))
('THE MOONFLOWER VINE', set(['Jetta Carleton']))
('THE CENTAUR', set(['John Updike']))


# Getting ISBN from Title/Author Name

In [19]:
import xml.etree.ElementTree
import urllib2
import re, string
%run "key.py"

suffixes = ['CPA', 'CSC', 'CSJ', 'DC', 'DD', 'DDS', 'DMD', 'DO', 'DVM', 'EDD', 'ESQ', 'II', 'III', 
            'IV', 'INC', 'JD', 'JR', 'LLD', 'LTD', 'MD', 'OD', 'OSB', 'PC', 'PE', 'PHD', 'RET', 
            'RGS', 'RN', 'RNC', 'SHCJ', 'SJ', 'SNJM', 'SR', 'SSMO', 'USA', 'USAF', 'USAFR', 'USAR', 
            'USCG', 'USMC', 'USMCR', 'USN', 'USNR']

digits = ['1','2','3','4','5','6','7','8','9','0']
symbol_dict = {'%':'percent','$':'dollar', '1/2':'half'}

isbn_plus_base_url = "https://api-2445581351187.apicast.io:443"
plus_key = isbn_plus_key
app_id = "7f7512d5"
search_plus = "/search?app_key="+plus_key+"&app_id="+app_id+"&t="
print isbn_plus_base_url+search_plus+"Potatoes"
alph = re.compile('[^a-zA-Z]')

manual_titles = []
db_key = isbn_db_key
isbn_db_base = "http://isbndb.com/api"
search_db = "/books.xml?access_key="+db_key+"&index1=title&value1="
print isbn_db_base+search_db+"Potatoes"

def convert_to_search(name):
    return_string = ""
    last_ch = " "
    for ch in name:
        if (ch == " "):
            if (last_ch != "."):
                return_string = return_string + "+"
        else:
            if ch == ".":
                return_string = return_string +".+"
            else:
                return_string = return_string+ch
        last_ch = ch
    return return_string

def url_to_xml(url):
    req = urllib2.Request(url)
    req.add_header('User-agent', 'Mozilla 5.10')
    res = urllib2.urlopen(req)
    print res
    data = xml.etree.ElementTree.parse(res).getroot()[0]
    for book in data.findall('BookData'):
        isbn = book.get('isbn')
        title = book.find('Title').text
        author = book.find('AuthorsText').text
        print(isbn,title,author)
    return data

def suffix(phrase):
    norm_phrase = phrase.replace(',',"").upper()
    if norm_phrase in suffixes:
        return True
    else:
        return False

def last_name_recursive(name,last_index):
    if last_index == 0:
        return ""
    last_phrase = name[last_index]
    if not suffix(last_phrase):
        return last_phrase
    else:
        return last_name_recursive(name, last_index-1)

def last_name(name):
    names = name.split(" ")
    last_index = len(names)-1
    last = last_name_recursive(names,last_index)
    return last

def strip_punct(text):
    return alph.sub('', text)

def has_numbers(text):
    for digit in digits:
        if digit in text:
            return True
    return False

def has_symbols(text):
    for symbol in symbol_dict:
        if symbol in text:
            return True
    return False

def norm_text(text):
    return strip_punct(text).lower()

def is_book(booxml, title, authors, year):
    xml_title = booxml.find('Title').text
    norm_xml_title = norm_text(xml_title)
    norm_title = norm_text(title)
    if(norm_xml_title in norm_title) or (norm_title in norm_xml_title):
        xml_authors = booxml.find('AuthorsText').text
        if xml_authors is None:
            return False
        for author in authors:
            if norm_text(author) in norm_text(xml_authors):
                print "Adding ISBN! "+xml_title+" by "+xml_authors
                return True
            xml_names = []
            for xmla in xml_authors.split(" "):
                norm_text(xmla)
            for author in authors:
                if norm_text(last_name(author)) in xml_authors.split(" "):
                    print "Adding ISBN! "+xml_title+" by "+xml_authors
                    return True        
        return False

def get_isbns(title,authors,year):
    isbns = set()
    search_t = convert_to_search(title)
    url = isbn_db_base+search_db+search_t
#    print row[0].year
    data = url_to_xml(url)
    for book in data:
        if (is_book(book,title,authors,year)):
            isbns.add(book.get('isbn'))
    return isbns

https://api-2445581351187.apicast.io:443/search?app_key=62f7cedf62afa7006026634aaadce211&app_id=7f7512d5&t=Potatoes
http://isbndb.com/api/books.xml?access_key=1DR9S9TA&index1=title&value1=Potatoes


In [20]:
for row in commasv:
    title_author = title_author_pair(row[2])
    print title_author
    if has_symbols(title_author[0]) or has_numbers(title_author[0]):
        manual_titles.append(title_author)
    else:
        isbns = get_isbns(title_author[0],title_author[1],1963)
        if len(isbns) == 0:
            manual_titles.append(title_author)
        else:
            print isbns


print manual_titles

('RAISE HIGH THE ROOF BEAM,  CARPENTERS', set(['J.D. Salinger']))
<addinfourl at 140366565609056 whose fp = <socket._fileobject object at 0x7fa9a347eb50>>
('0241950465', 'Raise High the Roof Beam, Carpenters', None)
('0553142828', 'Raise high the roof beam, carpenters', None)
('0140237518', 'Raise High the Roof Beam, Carpenters', 'J. D. Salinger, ')
('0141049243', 'Raise High the Roof Beam, Carpenters. Seymour', 'J.D. Salinger, ')
('055320596X', 'Raise high the roof beam, carpenters; and, Seymour', 'J. D. Salinger')
('0316769517', 'Raise High the Roof Beam, Carpenters and Seymour', 'J. D. Salinger, ')
('0140022643', 'Raise high the roof beam, carpenters; and Seymour', 'J. D. Salinger')
('0316766941', 'Raise high the roof beam, carpenters', None)
('0316769576', 'Raise high the roof beam, carpenters', None)
('0553125540', 'Raise high the roof beam, carpenters', 'J. D. Salinger')
Adding ISBN! Raise High the Roof Beam, Carpenters by J. D. Salinger, 
Adding ISBN! Raise High the Roof Beam, C

# Getting Wiki Gender Data from Author Name

In [53]:
wiki_name_to_id = "https://www.wikidata.org/w/api.php?action=wbsearchentities&language=en&format=json&search="
wiki_id_to_gender = "https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=claims&languages=en&ids="
gender_prop_id = "P21"
wiki_gender_hash = {"Q6581097":"male", "Q6581072":"female","Q1052281":"transf","Q2449503":"transm","Q1097630":"intersex"}

import urllib2
import json

def url_to_data( url ):
    req = urllib2.Request(url)
    req.add_header('User-agent', 'Mozilla 5.10')
    res = urllib2.urlopen(req)
    data = json.load(res)
    return data

def same_gender(id_list):
    id_0 = id_list[0]
    for i in id_list[1:]:
        if gender_from_id(id_0) != gender_from_id(i):
            return False
    return True

## This needs to disambiguate -- does it need to pull more information by id?
def disambiguate(entity_data, name):
    entities = []
    for entity in entity_data:
        if 'description' in entity:
            desc = entity['description'].lower()
            ent_id = entity['id']
            if ('novel' in desc) or ('write' in desc) or ('auth' in desc) or ('cartoo' in desc) or ('journ' in desc) or ('comic' in desc):
                entities.append((ent_id,desc))
    if len(entities)==0:
        print "No novelists, writers, authors, cartoonists, journalists or comics named "+name
        for x in entity_data:
            print x
    return entities

def get_wiki_id(name):
    search_term = convert_to_search(name)
    url = wiki_name_to_id + search_term
    data = url_to_data(url)['search']
    if data == []:
        return [-1]
    dis_data = disambiguate(data,name)
    if dis_data == []:
        return [-1]
    i = []
    for entity in dis_data:
        i.append(entity[0])
    return i

def gender_from_id(auth_id):
    url = wiki_id_to_gender + auth_id
    print url
    claims = url_to_data(url)['entities'][str(auth_id)]['claims']['P21']
    gender_arr =[]
    for claim in claims:
        if claim['mainsnak']['datavalue']['value'] is not None:
            gender_id = claim['mainsnak']['datavalue']['value']['id']
            gender_arr.append(wiki_gender_hash[gender_id])
    if len(gender_arr)>1:
        gender = max_value(gender_arr)
    else:
        gender = gender_arr[0]
    return gender


#def book_to_key_facts(book_data):
    
    

def author_to_gender(author_name):
    gender = ""
    author_id = []
    author_id = get_wiki_id(author_name)
    print author_id
    if author_id[0] == -1:
        print "Unable to find "+author_name+" in database"
        return "unknown"
    if len(author_id)>1:
        print "More than one writer by the name "+author_name
        if(same_gender(author_id)):
            gender = gender_from_id(author_id[0])
    else:
        gender = gender_from_id(author_id[0])
    return gender

In [51]:
print author_to_gender("Virginia Cary Hudson")

[u'Q15637611']
https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=claims&languages=en&ids=Q15637611
female


In [54]:
for row in commasv:
    authors = title_author_pair(row[2])[1]
    for author in authors:
        gender = author_to_gender(author)
        print author + " was a "+gender

[u'Q39212', u'Q5559244']
More than one writer by the name John Steinbeck
https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=claims&languages=en&ids=Q39212
https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=claims&languages=en&ids=Q5559244
https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=claims&languages=en&ids=Q39212
John Steinbeck was a male
[u'Q298920']
https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=claims&languages=en&ids=Q298920
Charles M. Schulz was a male
[u'Q353774']
https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&props=claims&languages=en&ids=Q353774
Adela Rogers St. Johns was a female
No novelists, writers, authors, cartoonists, journalists or comics named Hedda Hopper
{u'pageid': 262584, u'description': u'Actress, gossip columnist, radio personality', u'title': u'Q271324', u'concepturi': u'http://www.wikidata.org/entity/Q271324', u'label': u'Hedda Hopper', u'u